In [10]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from deap import base, creator, tools, algorithms


In [11]:
def recommendation_function(X_train):
    # Preprocess the data
    le = LabelEncoder()
    X_train['VehiclesTitle'] = le.fit_transform(X_train['VehiclesTitle'])
    X_train['FullName'] = le.fit_transform(X_train['FullName'])

    # Define the fitness function
    def evaluate(individual):
        score = 0
        for i in range(len(individual)):
            if individual[i] == 1:
                score += X_train.iloc[i]['PricePerDay'] * X_train.iloc[i]['Days']
        return (score,)

    # Create the individual class and population
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(X_train))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Initialize population
    population = toolbox.population(n=50)

    # Run the genetic algorithm
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)

    # Return the best individual
    best_individual = hof[0]
    recommendation = [X_train.iloc[i] for i, val in enumerate(best_individual) if val == 1]

    
    
    # Return only the top 10 recommendations
    return recommendation

In [12]:
def format_recommendations(recommendations):
        print(f"\n\nTop 5 Recommendations:\n")
        formatted_output = ""
        for rec in recommendations[:5]:
            
            formatted_output += "\n"
            formatted_output += f"VehicleId: {rec['VehicleId']}\n"
            formatted_output += f"VehiclesTitle: {rec['VehiclesTitle']}\n"
            formatted_output += f"PricePerDay(Npr): {rec['PricePerDay']:.2f}\n"
            formatted_output += f"id: {rec['id_y']}\n"

        return formatted_output

In [13]:
# Example usage
X_train = pd.read_csv("D:/WORK/test/major_proj_karuna/bikerental/data/test/X_test.csv")
y_train = pd.read_csv("D:/WORK/test/major_proj_karuna/bikerental/data/test/y_test.csv")
recommendations = recommendation_function(X_train)
# print("/n/nRecommendations:/n/n")
# for rec in recommendations:
#     print(rec)


c:\Users\Ripple\AppData\Local\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Ripple\AppData\Local\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg    	min      	max    
0  	50    	4.14901	-0.940162	9.54633
1  	38    	5.93451	2.26446  	9.54633
2  	39    	7.47136	3.11516  	10.8987
3  	41    	8.82553	4.91393  	12.2396
4  	37    	10.1107	7.48534  	12.28  
5  	46    	11.079 	8.61845  	12.28  
6  	34    	11.5842	9.5323   	12.28  
7  	41    	11.9782	9.48845  	12.28  
8  	37    	12.114 	9.66736  	12.28  
9  	39    	12.163 	10.6645  	12.28  
10 	37    	12.2092	10.804   	12.28  
11 	46    	12.1876	10.504   	12.28  
12 	41    	12.2278	10.504   	12.28  
13 	28    	12.0636	8.40709  	12.28  
14 	42    	12.1478	9.64193  	12.28  
15 	43    	12.1325	10.1059  	12.28  
16 	41    	12.1004	9.24025  	12.28  
17 	41    	11.9741	8.30097  	12.28  
18 	41    	12.0187	9.15957  	12.28  
19 	38    	12.1108	10.654   	12.28  
20 	48    	12.0613	9.51739  	12.28  
21 	35    	12.1563	9.13764  	12.28  
22 	35    	12.162 	10.9171  	12.28  
23 	38    	12.1066	9.81733  	12.28  
24 	44    	12.098 	9.29813  	12.28  
25 	41    	12.1683	10.654   	12.28  
2

In [14]:
# Print only specific columns from the top 10 recommended vehicles
print(format_recommendations(recommendations))



Top 5 Recommendations:


VehicleId: 26
VehiclesTitle: 12
PricePerDay(Npr): 1.65
id: 51

VehicleId: 33
VehiclesTitle: 10
PricePerDay(Npr): 1.10
id: 44

VehicleId: 4
VehiclesTitle: 8
PricePerDay(Npr): 0.33
id: 39

VehicleId: 18
VehiclesTitle: 4
PricePerDay(Npr): 1.36
id: 22

VehicleId: 1
VehiclesTitle: 0
PricePerDay(Npr): 0.38
id: 1

